In [ ]:
import sys
import os

parent = os.path.dirname(os.path.realpath("../"))
sys.path.append(parent)

import numpy as np
import scipy
import tqdm
import open3d as o3d
import matplotlib.pyplot as plt
import glob

from core import *
from utils import phantom_builder
from utils import geometry
from utils import utils

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Make Phantom

In [ ]:
voxel_size = np.array([0.0005, 0.0005, 0.0005])
surface_mesh = o3d.io.read_triangle_mesh(
    f"{parent}/assets/breast_phantom/breast_surface.ply"
)
body_mask = phantom_builder.voxelize(voxel_size[0], mesh=surface_mesh)

In [ ]:
test_phantom = phantom.Phantom(
    voxel_dims=(voxel_size[0], voxel_size[0], voxel_size[0]),
    matrix_dims=body_mask.shape,
    baseline=(1440, 1000),
    seed=None,
)

skin = tissue.Tissue(name="skin", c=1624, rho=1109, sigma=1.3, scale=0.00001, label=1)
coupling = tissue.Tissue(
    name="coupling", c=1440, rho=911, sigma=0, scale=0.00001, label=7
)
fat = tissue.Tissue(name="fat", c=1440, rho=911, sigma=40, scale=0.0003, label=2)
ligament = tissue.Tissue(
    name="ligament", c=1750, rho=1142, sigma=30, scale=0.0001, label=3
)
gland = tissue.Tissue(name="gland", c=1564, rho=1041, sigma=40, scale=0.0002, label=4)
tumor = tissue.Tissue(name="tumor", c=1560, rho=1050, sigma=0, scale=0.001, label=5)
muscle = tissue.Tissue(name="muscle", c=1580, rho=1090, sigma=15, scale=0.001, label=6)

kidney_file_dir = f"{parent}/assets/breast_phantom/"
kidney_tissue_list = [skin, fat, skin, gland, gland, ligament, muscle, tumor]
test_phantom.add_tissue(coupling)
test_phantom.build_organ_from_mesh(
    surface_mesh, voxel_size[0], kidney_tissue_list, dir_path=kidney_file_dir
)
# test_phantom.set_default_tissue('water')
test_phantom.set_default_tissue("coupling")

In [ ]:
test = test_phantom.get_complete()

In [ ]:
index = 110
plt.imshow(test[0, :, :, index], cmap="gray", vmin=1400, vmax=1650)
plt.imshow(
    body_mask[:, :, index] * 1000,
    alpha=0.5 * (body_mask[:, :, index] > 0) * 2,
    cmap="Reds",
)

# Set up simulation

In [ ]:
num_transducers = 8192
transducers = [
    transducer.Focused(
        max_frequency=1e6,
        elements=1,
        width=2e-3,
        height=2e-3,
        sensor_sampling_scheme="not_centroid",
        sweep=(0, 0),
        ray_num=(1, 1),
        imaging_ndims=3,
        focus_azimuth=float("inf"),
        focus_elevation=float("inf"),
        source_strength=1e7,
        cycles=1,
        transmit=i % 21 == 0,
    )
    for i in range(num_transducers)
]

for t in transducers:
    t.make_sensor_coords(1540)  # test_phantom.baseline[0]

test_transducer_set = transducer_set.TransducerSet(transducers, seed=8888)

In [ ]:
import math


def fibonacci_sphere(samples=1000, maxphi=2 * np.pi):
    coords = []
    angles = []
    phi = math.pi * (math.sqrt(5.0) - 1.0)
    for i in range(samples):
        x = 1 - (i / float(samples - 1)) * 2 * maxphi / (2 * np.pi)
        radius = math.sqrt(1 - x * x)
        theta = phi * i
        y = math.cos(theta) * radius
        z = math.sin(theta) * radius
        coords.append((x, y, z))
        angles.append((math.acos(x), -np.pi, theta % (np.pi * 2) - np.pi))
    return np.array(coords), np.array(angles)

In [ ]:
global_transform = geometry.Transform(
    [np.pi * 0.87, np.pi * -0.05, 0], [-0.00, 0.005, -0.01], intrinsic=False
)
coords, angles = fibonacci_sphere(num_transducers, maxphi=np.pi / 2)

for i, (coord, angle) in enumerate(zip(coords, angles)):
    test_transducer_set.assign_pose(
        i,
        global_transform
        * geometry.Transform(
            angle, global_transform.apply_to_point(coord) * 0.07, intrinsic=False
        ),
    )

In [ ]:
test_sensor = sensor.Sensor(
    transducer_set=test_transducer_set, aperture_type="extended_aperture"
)

In [ ]:
points = np.array(
    (
        o3d.io.read_triangle_mesh(
            f"{parent}/assets/breast_phantom/00_breast_single_VH_F_skin.obj"
        )
    )
    .sample_points_uniformly(1000)
    .points
)
points = points[:, [0, 1, 2]] - np.array(
    (
        (
            np.amax(np.array(surface_mesh.vertices)[:, 0])
            + np.amin(np.array(surface_mesh.vertices)[:, 0])
        )
        / 2,
        (
            np.amax(np.array(surface_mesh.vertices)[:, 1])
            + np.amin(np.array(surface_mesh.vertices)[:, 1])
        )
        / 2,
        (
            np.amax(np.array(surface_mesh.vertices)[:, 2])
            + np.amin(np.array(surface_mesh.vertices)[:, 2])
        )
        * -2,
    )
)

test_transducer_set.plot_transducer_coords(
    scale=0.1, phantom_coords=points, view=(0, 0)
)
test_transducer_set.plot_transducer_coords(
    scale=0.1, phantom_coords=points, view=(90, 90)
)
test_transducer_set.plot_transducer_coords(
    scale=0.1, phantom_coords=points, view=(0, 90)
)

In [ ]:
simprops = simulation.SimProperties(
    grid_size=(100e-3, 100e-3, 100e-3),
    voxel_size=(0.5e-3, 0.5e-3, 0.5e-3),
    PML_size=(8, 8, 8),
    PML_alpha=2,
    t_end=12e-5,  # [s]
    bona=6,  # parameter b/a determining degree of nonlinear acoustic effects
    alpha_coeff=0.5,  # [dB/(MHz^y cm)]
    alpha_power=1.5,
)

In [ ]:
test_experiment = experiment.Experiment(
    simulation_path="breast_synthetic_aperture",
    sim_properties=simprops,
    phantom=test_phantom,
    transducer_set=test_transducer_set,
    sensor=test_sensor,
    nodes=1,
    results=None,
    indices=None,
    workers=3,
    additional_keys=[],
)

test_experiment.save()

In [ ]:
test_experiment.plot_ray_path(0)

# Run Simulation

In [ ]:
test_experiment = experiment.Experiment.load("breast_synthetic_aperture")
test_experiment.run(dry=True, dry_fast=True)

In [ ]:
test_experiment.run(repeat=False)

# Reconstruction

In [ ]:
test_experiment = experiment.Experiment.load("breast_synthetic_aperture")
test_experiment.run(dry=True, workers=8, dry_fast=True)
test_experiment.add_results()

In [ ]:
test_reconstruction = reconstruction.Compounding(experiment=test_experiment)

In [ ]:
images2 = test_reconstruction.compound(
    workers=16,
    resolution_multiplier=1,
    combine=False,
    volumetric=True,
    attenuation_factor=16,
    save_intermediates=True,
)

In [ ]:
files = glob.glob("breast_synthetic_aperture/reconstruct/intermediate_image_00*.npz")
images1 = []
for path in tqdm.tqdm(files):
    images1.append(utils.load_array(path))
images1 = np.array(images1)

image1 = np.sum(images1, axis=0) / len(images1)

In [ ]:
utils.save_mrc(image1, "breast_synthetic_aperture.mrc")

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(10, 5))
index = 0.5
ax[0, 0].imshow(
    test[0, :, :, int(test.shape[3] * index)], cmap="gray", vmin=1400, vmax=1650
)
ax[1, 0].imshow(
    image1[:, :, int(image1.shape[2] * index)], cmap="gray", vmin=3000, vmax=15000
)
index = 0.55
ax[0, 1].imshow(
    test[0, :, int(test.shape[2] * index), :], cmap="gray", vmin=1400, vmax=1650
)
ax[1, 1].imshow(
    image1[:, int(image1.shape[1] * index), :], cmap="gray", vmin=3000, vmax=15000
)
index = 0.35
ax[0, 2].imshow(
    test[0, int(test.shape[1] * index), :, :], cmap="gray", vmin=1400, vmax=1650
)
ax[1, 2].imshow(
    image1[int(image1.shape[0] * index), :, :], cmap="gray", vmin=3000, vmax=15000
)